In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975649,24.0,10062431.0,15,9.0,8762.298856,0.0,9.0,0.0,0.0,0.00,...,2.0,6.0,15.0,1.0,2.0,30.0,62730.806297,0.0,2.0,1.0
32975650,24.0,10062431.0,21,1.0,14209.220918,0.0,1.0,0.0,0.0,0.00,...,2.0,6.0,15.0,1.0,2.0,30.0,62730.806297,0.0,2.0,1.0
32975651,24.0,10062431.0,other,1.0,6651.116446,0.0,1.0,0.0,0.0,0.00,...,2.0,6.0,15.0,1.0,2.0,30.0,62730.806297,0.0,2.0,1.0
32975652,24.0,10062431.0,36,22.0,8150.818314,22.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,2.0,30.0,62730.806297,0.0,2.0,1.0


In [2]:
df.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty

In [3]:
df['dt'].min(),df['dt'].max()

(1.0, 24.0)

# 嘗試建立模型 
# 訓練集是 X = dt[1~22] Y = dt[23] 
# 測試集是 X = dt[2~23] Y = dt[24]

# 由於過去消費排名 不一定等於未來消費排名 (例如 人如果結婚了 升遷了 換工作了 遍有錢了都會導致消費型態的改變)

# 因此除了考慮過去消費排名可能還要加入其他特徵

In [4]:
other_features = ['masts', #婚姻
'educd', #教育
'trdtp', #行業別
'naty', # 國籍
'poscd', # 職位別
'cuorg', #客戶來源
'slam', #正卡信用額度
'gender_code', #信別
'age', #年紀
'primary_card', #正附卡註記
] 

# 這裡的想法是上面特徵都會影響一個人的消費行為
# 例如正卡信用額度大的人 可能會在"奢侈品"上花很多錢 
# 或者年紀大的人可能會在"保健食品"上花很多錢
# 由於 上面這些特徵都會"隨時間改變" 因此只須"考慮離測試數據最接近的狀態"就好

In [5]:
df_sorted = df.sort_values(by='dt') # 考慮離測試數據最接近的狀態
chid_mapping = {}
for fname in tqdm(['slam','age']):
    chid_mapping[fname] = dict(zip(df_sorted['chid'],df_sorted[fname])) # chid到"特徵"的映射
chid_mapping['age']

100%|██████████| 2/2 [00:16<00:00,  8.48s/it]


{10321418.0: 4.0,
 10221236.0: 4.0,
 10023248.0: 3.0,
 10158504.0: 5.0,
 10442254.0: 5.0,
 10439008.0: 3.0,
 10143172.0: 5.0,
 10437736.0: 5.0,
 10218164.0: 5.0,
 10137616.0: 5.0,
 10205476.0: 5.0,
 10317914.0: 4.0,
 10209075.0: 5.0,
 10259075.0: 6.0,
 10385447.0: 7.0,
 10280827.0: 3.0,
 10255286.0: 4.0,
 10214189.0: 3.0,
 10250518.0: 3.0,
 10003931.0: 3.0,
 10490670.0: 3.0,
 10035298.0: 4.0,
 10273961.0: 4.0,
 10442158.0: 4.0,
 10003925.0: 3.0,
 10019878.0: 5.0,
 10082260.0: 4.0,
 10292430.0: 8.0,
 10141825.0: 5.0,
 10127255.0: 6.0,
 10184390.0: 5.0,
 10389484.0: 4.0,
 10438215.0: 5.0,
 10232793.0: 2.0,
 10211489.0: 4.0,
 10275313.0: 3.0,
 10488331.0: 5.0,
 10262058.0: 3.0,
 10156968.0: 3.0,
 10289942.0: 3.0,
 10106841.0: 3.0,
 10137542.0: 5.0,
 10002480.0: 5.0,
 10328172.0: 6.0,
 10114364.0: 2.0,
 10144624.0: 4.0,
 10123419.0: 4.0,
 10306732.0: 7.0,
 10384292.0: 5.0,
 10131507.0: 4.0,
 10358482.0: 2.0,
 10328323.0: 4.0,
 10479418.0: 4.0,
 10078621.0: 3.0,
 10299185.0: 6.0,
 10090666.

# create_x_y 函數

In [6]:
def create_x_y(df,split_dt,other_features,chid_mapping,mode='train'):
    官方指認欄位 = [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]
    官方指認欄位 = [str(i) for i in 官方指認欄位]
    origin_df = df
    def create_features(df,split_dt,look_back=3):
        df = df.loc[(df.dt>=split_dt-look_back)&(df.dt<=split_dt)] #根據dt做row的篩選
        df = df.pivot_table(index=['chid'], columns='shop_tag', values='txn_amt',aggfunc=np.sum).fillna(0.0) # 計算這個chid在各shop_tag的txn_amt
        df = df.loc[:,官方指認欄位] # 只取官方指認欄位
        df.columns = [ 'shop_tag_' + str(i) for i in df.columns]
        df['chid'] = df.index
        for fname in tqdm(other_features):
            df[fname] = df['chid'].apply(lambda x:chid_mapping[fname][x])
        return df
    x = create_features(df.loc[df.dt < split_dt,:],split_dt) # time = t-1 ,t-2 ... ,t-n
    y = create_features(df.loc[df.dt == split_dt,:],split_dt) # time = t
    common_index = set(x.index)&set(y.index) # 共同index
    x = x.loc[common_index]
    y = y.loc[common_index]
    print('x shape',x.shape)
    print('y shape',y.shape)
    return x,y

In [7]:
import gc
gc.collect()
locals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'os', 'sys', '_i', '_ii', '_iii', '_i1', 'pd', 'np', 'tqdm', 'df', '_1', '_i2', '_2', '_i3', '_3', '_i4', 'other_features', '_i5', 'df_sorted', 'chid_mapping', 'fname', '_5', '_i6', 'create_x_y', '_i7', 'gc'])

In [8]:
X_train, y_train = create_x_y(df,split_dt=23,other_features=['slam','age'],chid_mapping=chid_mapping,mode='train')
X_test, y_test = create_x_y(df,split_dt=24,other_features=['slam','age'],chid_mapping=chid_mapping,mode='test')

100%|██████████| 2/2 [00:00<00:00,  3.51it/s]


x shape (419727, 19)
y shape (419727, 19)


100%|██████████| 2/2 [00:00<00:00,  3.47it/s]


x shape (420732, 19)
y shape (420732, 19)


In [13]:
X_train

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,shop_tag_25,shop_tag_26,shop_tag_36,shop_tag_37,shop_tag_39,shop_tag_48,chid,slam,age
chid,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485760.0,102382.569589,6.0
10485761.0,42417.770262,0.000000,76522.964200,0.0,0.000000,27966.344293,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,14042.606023,0.000000,0.000000,11900.190480,10485761.0,102382.569589,3.0
10485762.0,15958.303518,0.000000,18527.238020,0.0,24371.585109,8677.196788,3480.859743,0.000000,0.000000,0.0,0.000000,0.000000,15579.865469,8135.177708,0.000000,0.000000,10485762.0,44545.625512,3.0
10485763.0,45026.598289,0.000000,0.000000,0.0,0.000000,20236.239795,3324.768457,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,25698.045779,24422.463317,39934.038243,10485763.0,99935.984396,4.0
10485764.0,0.000000,0.000000,0.000000,0.0,0.000000,10965.958583,0.000000,9277.143937,0.000000,0.0,0.000000,0.000000,0.000000,9779.293272,0.000000,0.000000,10485764.0,84815.219772,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485754.0,0.000000,0.000000,45937.403176,0.0,0.000000,5540.285984,10801.793485,0.000000,0.000000,0.0,39157.481379,15535.733355,0.000000,17398.744988,0.000000,5773.036485,10485754.0,87397.101503,3.0
10485755.0,34509.343013,0.000000,25676.391759,0.0,9958.409338,9373.392018,0.000000,0.000000,0.000000,0.0,0.000000,4219.341890,23365.474636,7829.247010,0.000000,11922.038035,10485755.0,38072.451849,2.0
10485756.0,0.000000,25004.783761,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485756.0,86109.761716,3.0


In [9]:
y_train

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,shop_tag_25,shop_tag_26,shop_tag_36,shop_tag_37,shop_tag_39,shop_tag_48,chid,slam,age
chid,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485760.0,102382.569589,6.0
10485761.0,0.000000,0.00000,10094.469556,0.000000,2688.055269,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,11647.892883,0.000000,0.000000,0.000000,10485761.0,102382.569589,3.0
10485762.0,11080.508913,0.00000,19347.923710,0.000000,9422.439888,7878.449264,0.000000,0.0,0.0,0.0,4404.325243,0.000000,10091.812993,0.000000,0.000000,0.000000,10485762.0,44545.625512,3.0
10485763.0,23260.509969,0.00000,0.000000,0.000000,0.000000,11504.412904,3900.970896,0.0,0.0,0.0,0.000000,41010.316376,0.000000,0.000000,0.000000,8019.906666,10485763.0,99935.984396,4.0
10485764.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,5773.036485,13515.952679,0.000000,0.000000,10485764.0,84815.219772,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485754.0,0.000000,0.00000,19175.333546,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,2411.854209,0.000000,5047.854508,0.000000,0.000000,10485754.0,87397.101503,3.0
10485755.0,6922.859869,0.00000,5744.585392,6303.494132,4497.402453,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,6603.592746,0.000000,0.000000,0.000000,10485755.0,38072.451849,2.0
10485756.0,0.000000,8334.92792,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485756.0,86109.761716,3.0


In [10]:
X_test

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,shop_tag_25,shop_tag_26,shop_tag_36,shop_tag_37,shop_tag_39,shop_tag_48,chid,slam,age
chid,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485760.0,102382.569589,6.0
10485761.0,12289.330042,0.000000,62643.191930,0.000000,2688.055269,22366.738610,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,24893.333242,0.000000,0.000000,11900.190480,10485761.0,102382.569589,3.0
10485762.0,27038.812431,0.000000,29267.566728,0.000000,24586.212696,16555.646052,0.000000,0.000000,0.000000,0.0,4404.325243,0.000000,23146.000680,8135.177708,0.000000,0.000000,10485762.0,44545.625512,3.0
10485763.0,52224.193152,0.000000,0.000000,0.000000,0.000000,22907.242157,7225.739353,0.000000,0.000000,0.0,0.000000,41010.316376,0.000000,21842.333126,7945.887166,37032.778347,10485763.0,99935.984396,4.0
10485764.0,0.000000,0.000000,0.000000,0.000000,0.000000,9331.575338,0.000000,9277.143937,0.000000,0.0,0.000000,0.000000,5773.036485,23295.245951,0.000000,0.000000,10485764.0,84815.219772,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485755.0,25003.263736,0.000000,31420.977151,6303.494132,9218.267691,2939.762251,0.000000,0.000000,0.000000,0.0,0.000000,4219.341890,22694.717135,0.000000,0.000000,4677.167185,10485755.0,38072.451849,2.0
10485756.0,0.000000,25004.783761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485756.0,86109.761716,3.0
10485757.0,36788.182809,0.000000,20688.378027,0.000000,13799.666117,47992.206162,12225.410860,6148.896046,25075.366933,0.0,9532.867791,70294.995919,0.000000,0.000000,21339.560666,0.000000,10485757.0,74152.141012,3.0


In [11]:
y_test

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,shop_tag_25,shop_tag_26,shop_tag_36,shop_tag_37,shop_tag_39,shop_tag_48,chid,slam,age
chid,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10485760.0,102382.569589,6.0
10485761.0,0.000000,0.00000,10173.700632,0.0,0.000000,13793.154923,0.000000,0.000000,0.00000,0.0,0.0,0.0,11010.627600,0.000000,0.000000,7675.133233,10485761.0,102382.569589,3.0
10485762.0,0.000000,0.00000,0.000000,0.0,5047.854508,0.000000,0.000000,14871.761480,0.00000,0.0,0.0,0.0,5021.010171,0.000000,0.000000,0.000000,10485762.0,44545.625512,3.0
10485763.0,0.000000,0.00000,0.000000,0.0,0.000000,14070.404760,2753.052186,0.000000,0.00000,0.0,0.0,0.0,0.000000,4684.483372,13995.306289,5773.036485,10485763.0,99935.984396,4.0
10485764.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,6257.629931,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10485764.0,84815.219772,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485755.0,8998.361272,0.00000,30379.591719,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,4243.989699,0.000000,0.000000,2120.034444,10485755.0,38072.451849,2.0
10485756.0,0.000000,8334.92792,0.000000,0.0,0.000000,4292.805774,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10485756.0,86109.761716,3.0
10485757.0,16084.241823,0.00000,0.000000,0.0,0.000000,20064.304813,13418.731765,0.000000,11997.68771,0.0,0.0,0.0,0.000000,0.000000,6308.563642,0.000000,10485757.0,74152.141012,3.0


# 由於Y應該是TOP3 這裡做一下轉換

In [14]:
def add_topk(df):
  var = np.argsort(np.array(df.values.tolist()))
  df[f'top{1}'] = var[:,-1]
  df[f'top{2}'] = var[:,-2]
  df[f'top{3}'] = var[:,-3]
  return df

In [15]:
y_train = add_topk(y_train)

In [17]:
y_train = y_train[['chid','top1','top2','top3']]
y_train

,chid,top1,top2,top3
chid,,,,
10485760.0,10485760.0,16,17,18
10485761.0,10485761.0,16,17,12
10485762.0,10485762.0,16,17,2
10485763.0,10485763.0,16,17,11
10485764.0,10485764.0,16,17,13
...,...,...,...,...
10485754.0,10485754.0,16,17,2
10485755.0,10485755.0,16,17,0
10485756.0,10485756.0,16,17,1


In [18]:
y_test = add_topk(y_test)

In [19]:
y_test = y_test[['chid','top1','top2','top3']]
y_test

,chid,top1,top2,top3
chid,,,,
10485760.0,10485760.0,16,17,18
10485761.0,10485761.0,16,17,5
10485762.0,10485762.0,16,17,7
10485763.0,10485763.0,16,17,5
10485764.0,10485764.0,16,17,7
...,...,...,...,...
10485755.0,10485755.0,16,17,2
10485756.0,10485756.0,16,17,1
10485757.0,10485757.0,16,17,5


In [20]:
X_train.head()

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,shop_tag_25,shop_tag_26,shop_tag_36,shop_tag_37,shop_tag_39,shop_tag_48,chid,slam,age
chid,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10485760.0,102382.569589,6.0
10485761.0,42417.770262,0.0,76522.96420,0.0,0.000000,27966.344293,0.000000,0.000000,0.0,0.0,0.0,0.0,14042.606023,0.000000,0.000000,11900.190480,10485761.0,102382.569589,3.0
10485762.0,15958.303518,0.0,18527.23802,0.0,24371.585109,8677.196788,3480.859743,0.000000,0.0,0.0,0.0,0.0,15579.865469,8135.177708,0.000000,0.000000,10485762.0,44545.625512,3.0
10485763.0,45026.598289,0.0,0.00000,0.0,0.000000,20236.239795,3324.768457,0.000000,0.0,0.0,0.0,0.0,0.000000,25698.045779,24422.463317,39934.038243,10485763.0,99935.984396,4.0
10485764.0,0.000000,0.0,0.00000,0.0,0.000000,10965.958583,0.000000,9277.143937,0.0,0.0,0.0,0.0,0.000000,9779.293272,0.000000,0.000000,10485764.0,84815.219772,4.0


In [21]:
X_test.head()

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,shop_tag_25,shop_tag_26,shop_tag_36,shop_tag_37,shop_tag_39,shop_tag_48,chid,slam,age
chid,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10485760.0,102382.569589,6.0
10485761.0,12289.330042,0.0,62643.191930,0.0,2688.055269,22366.738610,0.000000,0.000000,0.0,0.0,0.000000,0.000000,24893.333242,0.000000,0.000000,11900.190480,10485761.0,102382.569589,3.0
10485762.0,27038.812431,0.0,29267.566728,0.0,24586.212696,16555.646052,0.000000,0.000000,0.0,0.0,4404.325243,0.000000,23146.000680,8135.177708,0.000000,0.000000,10485762.0,44545.625512,3.0
10485763.0,52224.193152,0.0,0.000000,0.0,0.000000,22907.242157,7225.739353,0.000000,0.0,0.0,0.000000,41010.316376,0.000000,21842.333126,7945.887166,37032.778347,10485763.0,99935.984396,4.0
10485764.0,0.000000,0.0,0.000000,0.0,0.000000,9331.575338,0.000000,9277.143937,0.0,0.0,0.000000,0.000000,5773.036485,23295.245951,0.000000,0.000000,10485764.0,84815.219772,4.0


In [25]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(419727, 19) (420732, 19)
(419727, 4) (420732, 4)


In [27]:
X_train.reset_index(drop=True).to_feather('../data/X_train.feather')
y_train.reset_index(drop=True).to_feather('../data/y_train.feather')
X_test.reset_index(drop=True).to_feather('../data/X_test.feather')
y_test.reset_index(drop=True).to_feather('../data/y_test.feather')